In [1]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt

import multiscale.itk.metadata as meta
import multiscale.plotting as myplot
from multiscale.itk.itk_plotting import MultiImageDisplay, RegistrationPointDataAcquisition


In [2]:
us_path = Path(r'F:\Research\LINK\Phantom Trials\2018-12-07\Interleaved_5-1V_Sens0-6', 'Reslice of fused_tp_0_ch_0.tif')
mpm_path = Path(r'F:\Research\LINK\Phantom Trials\2018-12-04', 'MPM Grid shrunk 16x in xy.tif')

In [3]:
us_image = sitk.ReadImage(str(us_path))
us_image.SetSpacing([25, 25, 25])
mpm_image = sitk.ReadImage(str(mpm_path))
mpm_image.SetSpacing([16, 16, 7.5])

In [4]:
us_window = myplot.auto_window_level(sitk.GetArrayFromImage(us_image), return_image=False)
mpm_window = myplot.auto_window_level(sitk.GetArrayFromImage(mpm_image), bins=200, upper_limit_fraction=0.3, lower_limit_fraction=0.05, return_image=False)

In [5]:
MultiImageDisplay([us_image, mpm_image], title_list=['US', 'MPM'])

Box(children=(IntSlider(value=20, description='image slice:', max=40), IntSlider(value=0, description='image s…

FigureCanvasNbAgg()

In [11]:
points = RegistrationPointDataAcquisition(us_image, mpm_image)

FigureCanvasNbAgg()

In [12]:
fixed_image_points, moving_image_points = points.get_points_flat()
print(fixed_image_points)

[8757.9584453405, 8307.181227598569, 500.0, 13067.435147849463, 23887.59699820789, 500.0, 11658.567764336916, 21235.611335125453, 500.0, 19283.026545698925, 8804.428539426528, 500.0]


In [13]:

initial_transform = sitk.LandmarkBasedTransformInitializer(sitk.VersorRigid3DTransform(), 
                                                                fixed_image_points, 
                                                                moving_image_points)
print(initial_transform)

itk::simple::Transform
 VersorRigid3DTransform (000002BC872F7B70)
   RTTI typeinfo:   class itk::VersorRigid3DTransform<double>
   Reference Count: 1
   Modified Time: 1185
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.999057 -0.0434269 0 
     0.0434269 0.999057 0 
     0 0 1 
   Offset: [-5900.52, -3707.68, -500]
   Center: [13191.7, 15558.7, 500]
   Translation: [-6588.63, -3149.48, -500]
   Inverse: 
     0.999057 0.0434269 0 
     -0.0434269 0.999057 0 
     0 0 1 
   Singular: 0
   Versor: [ 0, 0, 0.0217186, 0.999764 ]



In [14]:
registered_image = sitk.Resample(mpm_image, us_image, initial_transform, sitk.sitkLinear, 0.0, mpm_image.GetPixelID())

In [15]:
MultiImageDisplay([us_image, registered_image], title_list=['US', 'MPM'])

Box(children=(IntSlider(value=20, description='image slice:', max=40), IntSlider(value=20, description='image …

FigureCanvasNbAgg()